<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_192Vectors_Group_Data_SectorEmbed_Normal_Loss_Debayan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! rm -rf /content/idl-spring-22-project-deepar
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

Cloning into 'idl-spring-22-project-deepar'...
remote: Enumerating objects: 1862, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1862 (delta 15), reused 0 (delta 0), pack-reused 1829
Receiving objects: 100% (1862/1862), 82.97 MiB | 24.95 MiB/s, done.
Resolving deltas: 100% (892/892), done.
Checking out files: 100% (1949/1949), done.


In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
! mkdir -p '/content/drive/MyDrive/DeepARExperiments/'

In [4]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 181 kB 67.1 MB/s 
     |████████████████████████████████| 144 kB 67.2 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4b045962ec4da5e879335c21e3619b90ae9bbcfcc99ef9d2c114640b98cb2693
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
import wandb

wandb.init(project="vector-training", entity="idl-deepar-spring-22", name="baseline-more-data-weighte-loss")

wandb: Currently logged in as: mmml (use `wandb login --relogin` to force relogin)


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [7]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_random_groups.py -gs=5 -r=50000 -w=192
! pip install kaleido

total rows:  0
total rows:  0
total rows:  19
total rows:  19
total rows:  526
total rows:  526
total rows:  526
total rows:  776
total rows:  879
total rows:  1656
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in subtract
  return (a - mns) / sstd
total rows:  1656
total rows:  2333
total rows:  2333
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in subtract
  return (a - mns) / sstd
found nans
total rows:  2333
total rows:  2384
total rows:  2384
total rows:  2814
total rows:  2858
total rows:  2869
total rows:  2869
total rows:  2879
total rows:  2879
total rows

In [ ]:
!python3 /content/idl-spring-22-project-deepar/src/data/visualize.py

tcmalloc: large alloc 1233903616 bytes == 0x55ddcb8ac000 @  0x7f5be213f615 0x55dd53e89057 0x55dd53f347b6 0x55dd53f32e59 0x55dd53e1e840 0x55dd53f91868 0x55dd53e7e49a 0x55dd53ef1464 0x55dd53eeaa2e 0x55dd53e7d88a 0x55dd53eec719 0x55dd53eeaa2e 0x55dd53e7d88a 0x55dd53eec719 0x55dd53eeaa2e 0x55dd53eea723 0x55dd53fb4812 0x55dd53fb4b8d 0x55dd53fb4a36 0x55dd53f8c183 0x55dd53f8be2c 0x7f5be0f28c87 0x55dd53f8bd0a
^C


In [8]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/sector_company_map.pkl','rb') as f:
  sector_company_map = pickle.load(f)

In [9]:
sector_to_index = {}
index_to_sector = {}

for i, sector_name in enumerate(sorted(sector_company_map.keys())):
    sector_to_index[sector_name]  = i
    index_to_sector[i] = sector_name

print(sector_to_index)
print(index_to_sector)

{'Basic Materials': 0, 'Communication Services': 1, 'Consumer Cyclical': 2, 'Consumer Defensive': 3, 'Energy': 4, 'Financial Services': 5, 'Healthcare': 6, 'Industrials': 7, 'Real Estate': 8, 'Services': 9, 'Technology': 10, 'Utilities': 11}
{0: 'Basic Materials', 1: 'Communication Services', 2: 'Consumer Cyclical', 3: 'Consumer Defensive', 4: 'Energy', 5: 'Financial Services', 6: 'Healthcare', 7: 'Industrials', 8: 'Real Estate', 9: 'Services', 10: 'Technology', 11: 'Utilities'}


In [10]:
print(index_to_company)
print(company_to_index)
print(sector_company_map)
print(sector_company_map.keys())
print(sector_company_map['Technology'])
print(len(sector_company_map))

{0: 'LQMT', 1: 'LMT', 2: 'R', 3: 'APT', 4: 'PL', 5: 'SKIN', 6: 'TQLB', 7: 'MO', 8: 'ACI', 9: 'TGT', 10: 'SUNL', 11: 'L', 12: 'AMP', 13: 'MA', 14: 'PSLV', 15: 'KO', 16: 'COTY', 17: 'FAMI', 18: 'MNST', 19: 'D', 20: 'PCG', 21: 'ES', 22: 'HLGN', 23: 'OEG', 24: 'AGR', 25: 'ONE', 26: 'CLX', 27: 'COST', 28: 'WMT', 29: 'SBEV', 30: 'CRWD', 31: 'NVDA', 32: 'ZI', 33: 'DOCU', 34: 'ENPH', 35: 'EXAS', 36: 'VBIV', 37: 'EXEL', 38: 'CLPT', 39: 'CVS', 40: 'ATAI', 41: 'CDNA', 42: 'SNDL', 43: 'TGGI', 44: 'NURO', 45: 'Z', 46: 'CNK', 47: 'EA', 48: 'SNAP', 49: 'GOOG', 50: 'VINE', 51: 'FREE', 52: 'PG', 53: 'CELH', 54: 'AG', 55: 'CLF', 56: 'NEM', 57: 'AFPW', 58: 'AMRS', 59: 'FSR', 60: 'JD', 61: 'RKUNY', 62: 'LCID', 63: 'RENN', 64: 'PDD', 65: 'NEGG', 66: 'PZZA', 67: 'BBIG', 68: 'DKS', 69: 'BLUE', 70: 'ATIP', 71: 'ERBB', 72: 'AFMD', 73: 'RWBYF', 74: 'PINS', 75: 'DASH', 76: 'TTWO', 77: 'SNPW', 78: 'ATVI', 79: 'PLD', 80: 'AMT', 81: 'OPAD', 82: 'AHT', 83: 'MSTO', 84: 'VFF', 85: 'GLASF', 86: 'INMD', 87: 'BNTX', 88: 

In [11]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [12]:
NUM_TRAIN_SAMPLES = len(stock_inputs)
NUM_TEST_SAMPLES  = len(stock_test_inputs)

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [13]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (50409, 192, 13)
stock_labels_trimmed shape (50409, 192, 13)
stock_test_inputs_trimmed shape (13654, 192, 13)
stock_test_labels_trimmed shape (13654, 192, 13)


In [14]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

def plot_data(data, window_id,cov_id):
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  df = pd.DataFrame()
  df['data'] = data[window_id,:,cov_id]
  

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['data'], mode = 'lines', name = 'data'))

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Y')
  fig.show()

In [15]:
plot_data(stock_inputs_trimmed, 20000, 12)

In [30]:


params = {
    'num_classes': len(sector_company_map),
    'embedding_dim':len(sector_company_map),
    #  month, day, 5 companies open and volume
    'cov_dim': 7,
    'op_dim': 5,  # op does not have the "time covariates"
    'lstm_hidden_dim': 64,
    'lstm_layers':4,
    'window_size':192,
    'batch_size': 64,
    'learning_rate': 0.005,
    'epochs':100,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'num_companies': 5,
    'mu_sigma_dim': 1,  # each datapoint is one-d,
    'scheduler_step_size': 20,
    'scheduler_gamma': 0.95
}
# wandb.config = params
print(params)

{'num_classes': 12, 'embedding_dim': 12, 'cov_dim': 7, 'op_dim': 5, 'lstm_hidden_dim': 64, 'lstm_layers': 4, 'window_size': 192, 'batch_size': 64, 'learning_rate': 0.005, 'epochs': 100, 'num_test_samples': 13654, 'num_train_samples': 50409, 'conditioning_period': 168, 'prediction_period': 24, 'num_companies': 5, 'mu_sigma_dim': 1, 'scheduler_step_size': 20, 'scheduler_gamma': 0.95}


In [31]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs
        self.label = stock_labels
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
        x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))

        win_len, ip_dim = x1.shape

        sector_name = self.data[index][0,12]
        sector_id = sector_to_index[sector_name]

        sector_id_concat = sector_id * torch.ones((win_len, 1))
        x1 = torch.cat((x1, sector_id_concat), dim=1)
        return x1, x2  

class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs
        self.label = stock_test_labels
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
      x1 = torch.from_numpy(self.data[index][:,[0,2,4,6,8,10,11]].astype(np.float32))
      x2 = torch.from_numpy(self.label[index][:,[0,2,4,6,8]].astype(np.float32))

      # Add Sector ID to the input
      win_len, ip_dim = x1.shape

      sector_name = self.data[index][0,12]  # 12 is company index id
      sector_id = sector_to_index[sector_name]

      # sector_id_concat -> Sector Id tensor that is to be concatenated to x1
      sector_id_concat = sector_id * torch.ones((win_len, 1))
      x1 = torch.cat((x1, sector_id_concat), dim=1)

      return x1, x2  

In [32]:
train_dataset = TrainDataset()

for x1, x2 in train_dataset:

    print("data shape ")
    print(x1[:,-1])
    break


data shape 
tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])


In [33]:
train_data = TrainDataset()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)


In [34]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sqrt((1/(N)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result


# MODEL


In [35]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        
        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )

        self.price_mu = nn.Sequential(
                        nn.Linear(params['lstm_hidden_dim'], params['num_companies']),
                        nn.GELU()
                      )
        
        self.price_sigma = nn.Sequential(
                        nn.Linear(params['lstm_hidden_dim'], params['num_companies']),
                        nn.Softplus()
                      )
        
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])
        

    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''

        cov = x[:, :, :-1]   # remove the sector index from the inputs to get the covariates
        sector_index = x[:, 0, -1].to(torch.int32)  # retrieve the sector index from the covariates
        onehot_embed = self.embedding(sector_index)

        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_price_mu = self.price_mu(out1)
        out_price_sigma = self.price_sigma(out1)
        
        return out_price_mu, out_price_sigma, hn_cn



In [36]:
model = Network(params)
print(model)

Network(
  (lstm): LSTM(19, 64, num_layers=4, batch_first=True)
  (price_mu): Sequential(
    (0): Linear(in_features=64, out_features=5, bias=True)
    (1): GELU()
  )
  (price_sigma): Sequential(
    (0): Linear(in_features=64, out_features=5, bias=True)
    (1): Softplus(beta=1, threshold=20)
  )
  (embedding): Embedding(12, 12)
)


In [24]:
for ip, op in train_loader:
    print("ip shape", ip.shape)
    print("op shape", op[0].shape)

    fwd = model.forward(ip)
    print("fwd shape", fwd[0].shape)
    break

ip shape torch.Size([64, 192, 8])
op shape torch.Size([192, 5])
fwd shape torch.Size([64, 192, 5])


In [25]:
def gauss_fit_and_sample(price_mu, price_sigma):
  window_len = price_mu.shape[1]
  batch_size = price_mu.shape[0]

  # BATCH, SEQ_LEN, NUM_COMPANIES, 1
  price_mu_reshape = price_mu.view(batch_size * window_len * params['num_companies'], params['mu_sigma_dim'])
  price_sigma_reshape = price_sigma.view(batch_size * window_len * params['num_companies'], params['mu_sigma_dim'])

  gauss_price = torch.distributions.Normal(price_mu_reshape, price_sigma_reshape)

  # BATCH, SEQ_LEN, NUM_COMPANIES, 1
  price_sampled_reshape = gauss_price.sample()
  price_sampled = price_sampled_reshape.view(batch_size, window_len, params['num_companies'])

  return price_sampled

In [26]:
mu_rand = torch.rand(2, 3, 5)
sigma_rand = torch.ones((2,3,5))* 0.000000001

sampled_data = gauss_fit_and_sample(mu_rand, sigma_rand)

print(mu_rand)
print(sampled_data)


tensor([[[0.0796, 0.3626, 0.4163, 0.4352, 0.3474],
         [0.1266, 0.1493, 0.5315, 0.5452, 0.9812],
         [0.8587, 0.5068, 0.1878, 0.2545, 0.4619]],

        [[0.2278, 0.4329, 0.7757, 0.7743, 0.9720],
         [0.9311, 0.8849, 0.1754, 0.7811, 0.7559],
         [0.3709, 0.1432, 0.8298, 0.1921, 0.5188]]])
tensor([[[0.0796, 0.3626, 0.4163, 0.4352, 0.3474],
         [0.1266, 0.1493, 0.5315, 0.5452, 0.9812],
         [0.8587, 0.5068, 0.1878, 0.2545, 0.4619]],

        [[0.2278, 0.4329, 0.7757, 0.7743, 0.9720],
         [0.9311, 0.8849, 0.1754, 0.7811, 0.7559],
         [0.3709, 0.1432, 0.8298, 0.1921, 0.5188]]])


In [27]:
.0# Test Code Conditioning Period + Prediction Period


def validate(model):

  model.eval()


  conditionining_period = params['conditioning_period']
  prediction_period     = params['prediction_period']

  assert conditionining_period + prediction_period == params['window_size']

  with torch.no_grad():
      
    overall_price_mu = None
    # overall_volume_mu = None

    overall_price_sigma = None
    # overall_volume_sigma = None

    overall_price_label = None
    # overall_volume_label = None

    overall_price_pred = None
    # overall_volume_pred = None

    for i, (ip_covariate, op_label) in enumerate(test_loader):

      ip_covariate = ip_covariate.cuda()
      op_label = op_label.cuda()

      cond_ip = ip_covariate[:, 0:params['conditioning_period'], :]
      cond_op = op_label[:, 0:params['conditioning_period'], :]

      pred_ip = ip_covariate[:, params['conditioning_period']:, :]
      pred_op = op_label[:, params['conditioning_period']:, :]

      # Step One - Forward Pass : Conditioning Period.
      price_mu, price_sigma, ht_ct = model.forward(x=cond_ip, h0_c0=None)

      # Sample GMM and get PRICE/ volume
      price_sampled = gauss_fit_and_sample(price_mu, price_sigma)

      price_sampled_last = price_sampled[:, -1, :].unsqueeze(1)

      batch_price_mu = price_mu
      batch_price_sigma = price_sigma

      batch_price_pred = price_sampled

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)

        # pred_cov_ip[:, 0, 2:2 + params['num_companies']] = price_sampled_last[:, 0, :]
        pred_cov_ip[:, 0, 0:params['num_companies']] = price_sampled_last[:, 0, :]
        price_pred_mu, price_pred_sigma, ht_ct = model.forward(x=pred_cov_ip,h0_c0=ht_ct)
        price_sampled_last = gauss_fit_and_sample(price_pred_mu, price_pred_sigma)

        batch_price_mu = torch.cat((batch_price_mu, price_pred_mu), dim=1)
        batch_price_sigma = torch.cat((batch_price_sigma, price_pred_sigma), dim=1)        
        batch_price_pred = torch.cat((batch_price_pred, price_sampled_last), dim=1)

      overall_price_mu = batch_price_mu if overall_price_mu is None else torch.cat((overall_price_mu, batch_price_mu), dim=0)
      overall_price_sigma = batch_price_sigma if overall_price_sigma is None else torch.cat((overall_price_sigma, batch_price_sigma), dim=0)
      overall_price_pred = batch_price_pred if overall_price_pred is None else torch.cat((overall_price_pred, batch_price_pred), dim=0)
      overall_price_label = op_label[:, :, 0:params['num_companies']] if overall_price_label is None else torch.cat((overall_price_label, op_label[:, :, 0:params['num_companies']]), dim=0)
    
  
  return overall_price_pred,overall_price_label, overall_price_mu, overall_price_sigma


In [28]:
def loss_fn(out_price_mu, out_price_sigma, op_label):

    B, S, D = out_price_mu.shape

    out_price_mu = out_price_mu.view(B * S * params['num_companies'], params['mu_sigma_dim'])
    out_price_sigma = out_price_sigma.view(B * S * params['num_companies'], params['mu_sigma_dim'])
    
    gauss_price =  torch.distributions.Normal(out_price_mu, out_price_sigma)
    
    op_label_price = op_label.view(B * S * params['num_companies'], params['mu_sigma_dim'])

    sample_loss =  -1 * torch.sum(gauss_price.log_prob(op_label_price)) / (B * S * params['num_companies'])

    return sample_loss


In [29]:
def loss_fn_v2(out_price_mu, out_price_sigma, op_label):

    B, S, D = out_price_mu.shape

    loss = 0
    for b in range(0, B):
      for seq in range(0, S):
        mean = out_price_mu[b, seq].view(-1, 1)
        sigma = out_price_sigma[b, seq].view(-1, 1)
        op    = op_label[b, seq].view(-1, 1)

        gauss =  torch.distributions.Normal(mean, sigma)
        # print("mean", mean.shape, "sigma", sigma.shape, "op", op.shape)
        loss += -1 * torch.sum(gauss.log_prob(op))
    return  loss / (B * S * params['num_companies'])



In [ ]:
mu_rand = torch.rand(2, 3, 5)
sigma_rand = torch.rand(2, 3, 5)
op_rand = torch.randn(2, 3, 5)

loss_val = loss_fn(mu_rand, sigma_rand, op_rand)
loss_val_v2 = loss_fn_v2(mu_rand, sigma_rand, op_rand)

print("loss fn ", loss_val, "loss val v2 ", loss_val_v2)

sigma_rand = 0.000000001 * torch.ones((2, 3, 4))


# print("mu shape", mu_rand.shape)

loss fn  tensor(18.5288) loss val v2  tensor(18.5288)


In [ ]:
model = Network(params)

model_version='vectorized_gaussian_normal_loss_sector_embed_V1.pt'
epochs = 100

optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=params['scheduler_step_size'], gamma=params['scheduler_gamma'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, threshold=0.01, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08, verbose=False)

for epoch in range(0, epochs):

  batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
  model.train()
  model.cuda()

  total_loss = 0
  total_price_loss = 0
  total_volume_loss = 0

  if os.path.exists(f'/content/drive/MyDrive/DeepARExperiments/{model_version}'):
      # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

      checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      epoch = checkpoint['epoch'] + 1
  
  for i, (ip_covariate, op_label) in enumerate(train_loader):

    optimizer.zero_grad()

    ip_covariate = ip_covariate.cuda()
    op_label = op_label.cuda()
    
    out_price_mu, out_price_sigma, ht_ct = model.forward(x=ip_covariate)
    
    batch_size = out_price_mu.shape[0]
    seq_with_horizon = params['window_size']

    out_price_mu = out_price_mu.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])
    out_price_sigma = out_price_sigma.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])

    loss = 0


    op_label_price = op_label.view(batch_size * seq_with_horizon * params['num_companies'], params['mu_sigma_dim'])
    gauss_price =  torch.distributions.Normal(out_price_mu, out_price_sigma)   

    loss =  -1 * torch.sum(gauss_price.log_prob(op_label_price)) / (batch_size * seq_with_horizon * params['num_companies'])

    loss.backward()

    total_loss += float(loss)
    optimizer.step()

    batch_bar.set_postfix(
    loss="{:.04f}".format(float(total_loss / (i + 1))),
    lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

    batch_bar.update() 

  batch_bar.close() # You need this to close the tqdm bar
  overall_price_pred, overall_price_label, overall_price_mu, overall_price_sigma = validate(model)
  scheduler.step(loss)

  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': total_loss/len(train_loader),
          },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')


  rmse_price = rmse(overall_price_pred.view(-1, 1), overall_price_label.view(-1, 1))
  nd_price = ND(overall_price_pred.view(-1, 1), overall_price_label.view(-1, 1))
  wandb.log({"loss":loss})
  print(f"Epoch {epoch}, price loss {loss}")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# def plot_test_output_data(actual,predicted, window_id,cov_id):
#   x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
#   df = pd.DataFrame()
#   df['actual'] = actual[window_id,:,cov_id]
#   df['predicted'] = predicted[window_id,:,cov_id]
  

#   fig = go.Figure()
#   fig.add_vline(x=168)
#   fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
#   fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

#   fig.update_xaxes(title_text='Time Samples')
#   fig.update_yaxes(title_text='Normalized Stock Opening Price')
#   fig.show()
  


In [56]:
def plot_test_output_data(actual,predicted, sigma, cov_id, window_id):

  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id, :, cov_id]
  df['predicted'] = predicted[window_id, :, cov_id]

  df['sigma'] = sigma[window_id, :, cov_id]
  #df['sampled']=sampled[window_id]
  df['predicted_upper'] =   df['predicted'] + 1*df['sigma']
  df['predcited_lower'] =   df['predicted'] - 1*df['sigma']
  # print("df predicted = ", df['predicted'])

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()

In [68]:
for i in range(5):
  plot_test_output_data(overall_price_label.detach().cpu(),overall_price_mu.detach().cpu(), overall_price_sigma.detach().cpu(), window_id=8000, cov_id=i)